## SETUP

In [ ]:
#Must be in the "ADAPET" Directory
%cd ADAPET

In [18]:
#Installing required libraries
!pip install -r requirements.txt

In [5]:
## Setting up environment variables
%env ADAPET_ROOT= /content/drive/MyDrive/ADAPET
%env PYTHONPATH=$ADAPET_ROOT:$PYTHONPATH
%env PYTHON_EXEC=python

env: ADAPET_ROOT=/content/drive/MyDrive/ADAPET
env: PYTHONPATH=$ADAPET_ROOT:$PYTHONPATH
env: PYTHON_EXEC=python


In [19]:
## Delete previous experiment outputs
# !rm -r exp_out

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report

## DATASET

In [23]:
## Data file placed in the parent directory of "ADAPET"
df = pd.read_excel("../PE_combined.xlsx")

## Data proccessing
df.drop(columns=['Unnamed: 0','Accession Number'],inplace=True)
df['doc'] = df['doc'].replace(r'\s+|\\n', ' ', regex=True) 
df['target'] = df['target'].replace({'Definitive PE NEG':'0'})
df['target'] = df['target'].replace({'Definitive PE POS':'2'})
df['target'] = df['target'].replace({'Probable PE NEG':'1'})
df['target'] = df['target'].replace({'Probable PE POS':'1'})
df['target'] = df['target'].replace({'Indeterminate':'inconclusive'})
df['target'] = df['target'].replace({'Non-diagnostic':'inconclusive'})
df = df[df['target'] != 'inconclusive']

In [26]:
## random_state must be same with other models.
train_df,test_df = train_test_split(df, test_size=0.2,stratify=df['target'],random_state=15)

In [11]:
pd.options.mode.chained_assignment = None  # default='warn'

train_df.rename(columns={'target':'LBL','doc':'TEXT1'},inplace=True)
train_df['LBL'].replace({'0':'negative'},inplace=True)
train_df['LBL'].replace({'1':'probable'},inplace=True)
train_df['LBL'].replace({'2':'positive'},inplace=True)

test_df.rename(columns={'target':'LBL','doc':'TEXT1'},inplace=True)
test_df['LBL'].replace({'0':'negative'},inplace=True)
test_df['LBL'].replace({'1':'probable'},inplace=True)
test_df['LBL'].replace({'2':'positive'},inplace=True)


In [12]:
train_df

,TEXT1,LBL
55,1. No evidence of pulmonary embolism. 2. Multi...,negative
70,1. No pulmonary embolism. Enlargement of the c...,negative
106,1.No definite evidence of pulmonary embolism. ...,probable
74,1. No evidence of pulmonary embolism. 2. No fo...,negative
150,Suboptimal bolus timing for evaluation of the ...,probable
...,...,...
33,1. No pulmonary embolism. 2. Findings of CHF i...,negative
24,1. No pulmonary embolism. Enlargement of the c...,negative
110,1. No pulmonary embolism. 2. Mildly enlarged m...,negative
65,No evidence of central pulmonary embolism. The...,probable


In [ ]:
## You will need GPU to train ADAPET model.  
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c2e6525e-6404-4205-c5c2-2499a82d5f9b)


## PROMPT SEARCH
We tested four different prompts: ["The diagnosis is", "Pulmonary embolism is", "Existence of pulmonary embolism is", "PE diagnosis is"]

i.e. We train the model on '[TEXT1]  [PROMPT]  [LBL]' where [TEXT1] is the input, [PROMPT] is the selected prompt, and [LBL] is the correct label. 

For each prompt, we perform 5-fold cross validation on the train set. We choose the prompt that resulted in the best macro average score. 

Note that Since the ADAPET codebase uses scripts, the prompt search in this notebook requires a lot of manual lines. We've only included code for testing "The diagnosis is". 

You can change the parameters "pattern" and "dir_names" (where the trained model is saved).


In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
i = 0
for train_index, eval_index in skf.split(train_df['TEXT1'], train_df['LBL']):
    t = train_df.iloc[train_index]
    e = train_df.iloc[eval_index]
    t.to_json('data/my_task/train.jsonl',lines=True,orient='records')
    
    e.to_json('data/my_task/test.jsonl',lines=True,orient='records')
    if i == 0:
        !python cli.py --data_dir data/my_task --pattern '[TEXT1] The diagnosis is [LBL]'  --dict_verbalizer '{"negative": "negative", "probable": "probable","positive":"positive"}' --num_batches 50  --dir_names 'v1f1'
    elif i == 1:
        !python cli.py --data_dir data/my_task --pattern '[TEXT1] The diagnosis is [LBL]'  --dict_verbalizer '{"negative": "negative", "probable": "probable","positive":"positive"}' --num_batches 50 --dir_names 'v1f2'
    elif i == 2:
        !python cli.py --data_dir data/my_task --pattern '[TEXT1] The diagnosis is [LBL]'  --dict_verbalizer '{"negative": "negative", "probable": "probable","positive":"positive"}' --num_batches 50 --dir_names 'v1f3'
    elif i == 3:
        !python cli.py --data_dir data/my_task --pattern '[TEXT1] The diagnosis is [LBL]'  --dict_verbalizer '{"negative": "negative", "probable": "probable","positive":"positive"}' --num_batches 50 --dir_names 'v1f4'
    elif i == 4:
        !python cli.py --data_dir data/my_task --pattern '[TEXT1] The diagnosis is [LBL]'  --dict_verbalizer '{"negative": "negative", "probable": "probable","positive":"positive"}' --num_batches 50  --dir_names 'v1f5'
    i+= 1


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
Downloading: 100% 710/710 [00:00<00:00, 628kB/s]
Downloading: 100% 760k/760k [00:00<00:00, 2.20MB/s]
Downloading: 100% 1.31M/1.31M [00:00<00:00, 2.55MB/s]
Downloading: 100% 893M/893M [00:08<00:00, 103MB/s]
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd


In [ ]:
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/v1f1/
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/v1f2/
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/v1f3/
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/v1f4/
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/v1f5/
saves = ['v1f1','v1f2','v1f3','v1f4','v1f5',]
import pandas as pd
from sklearn.metrics import classification_report
for save in saves:
    predictions = pd.read_json(f"exp_out/generic/albert-xxlarge-v2/{save}/test.json",lines=True)
    truth = pd.read_json("data/my_task/test.jsonl",lines=True)
    print(classification_report(list(truth['LBL']),list(predictions['label'])))

## TEST

We found "Pulmonary embolism is" to be the best prompt.

Note that because we did cv with train set and found the best prompt, we do not have evaluation set. ADAPET codebase by default requires an eval dataset. To work around this, we set the value for eval_every in the config file to be the same as the num_batches hyperparameter (50)

In [27]:
train_df.to_json('data/my_task/train.jsonl',lines=True,orient='records')
test_df.to_json('data/my_task/test.jsonl',lines=True,orient='records')
!python cli.py --data_dir data/my_task --pattern '[TEXT1] Pulmonary embolism is [LBL]'  --dict_verbalizer '{"negative": "negative", "probable": "probable","positive":"positive"}' --num_batches 50  --dir_names 't1'

In [ ]:
!sh bin/test.sh exp_out/generic/albert-xxlarge-v2/t1/
predictions = pd.read_json(f"exp_out/generic/albert-xxlarge-v2/t1/test.json",lines=True)
truth = pd.read_json("data/my_task/test.jsonl",lines=True)
print(classification_report(list(truth['LBL']),list(predictions['label'])))

+ exp_dir=exp_out/generic/albert-xxlarge-v2/t1/
+ python -m src.test -e exp_out/generic/albert-xxlarge-v2/t1/
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
              precision    recall  f1-score   support

    negative       1.00      0.95      0.97        20
    positive       0.67      0.80      0.73         5
    probable       0.83      0.83      0.83         6

    accuracy                           0.90        31
   macro avg       0.83      0.86      0.84        31
weighted avg       0.91      0.90      0.91        31

